In [1]:
# Extract 3-hourly 500 hPa Relative Humidity from regridded vertical velocity (hur) data of UM zoom 6 model
# Extract 6-ourly 500 hPa Relative Humidity from regridded vertical velocity (hur) data of ICON zoom 6 model
import xarray as xr
from pathlib import Path
import easygems.healpix as egh
import healpy as hp
import numpy as np

p = 500 # hPa; level of interest
data = "/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT3H.z6.zarr" #PT3H is 3-hourly data
data = "/g/data/qx55/germany_node/d3hp003.zarr/PT6H_point_z6_atm.zarr" #PT6H is 6-hourly data
ds = xr.open_zarr(data)

#r = ds['hur'].sel(pressure=p)
r = ds['hur'].sel(pressure=p*100)

def get_nn_lon_lat_index(nside, lons, lats):
    """
    nside: integer, power of 2. The return of hp.get_nside()
    lons: uniques values of longitudes
    lats: uniques values of latitudes
    returns: array with the HEALPix cells that are closest to the lon/lat grid
    """
    lons2, lats2 = np.meshgrid(lons, lats)
    return xr.DataArray(
        hp.ang2pix(nside, lons2, lats2, nest = True, lonlat = True),
        coords=[("latitude", lats), ("longitude", lons)],
    )

In [2]:
this_nside = hp.get_nside(r)
res = 1  # resolution
#MC lon: 100 - 149E, lat: 13S to 13N
lon = np.arange(100, 149 + res, res)
lat = np.arange(-13, 13 + res, res)

cells = get_nn_lon_lat_index(this_nside, lon, lat) 
r_regrid = r.isel(cell=cells)
r_regrid

<xarray.DataArray 'hur' (time: 1700, latitude: 27, longitude: 50)> Size: 9MB
[2295000 values with dtype=float32]
Coordinates:
    pressure   int64 8B 50000
  * time       (time) datetime64[ns] 14kB 2020-01-01T06:00:00 ... 2021-03-01
  * latitude   (latitude) int64 216B -13 -12 -11 -10 -9 -8 ... 8 9 10 11 12 13
  * longitude  (longitude) int64 400B 100 101 102 103 104 ... 146 147 148 149
Attributes:
    grid_mapping:        crs
    hiopy::time_method:  point
    hiopy::src_name:     pl::hur
    hiopy::nnn:          4
    hiopy::enable:       True
    standard_name:       relative_humidity
    units:               
    short_name:          
    long_name:           relative humidity

In [3]:
import pandas as pd
# Create full date range from 2020-03-01 to 2021-02-28
full_range = pd.date_range(start="2020-03-01", end="2021-02-28", freq="3h")
full_range = pd.date_range(start="2020-03-01", end="2021-02-28", freq="6h")

# Define exclusion period
exclude_start = pd.Timestamp("2020-05-01 00:00:00")
exclude_end = pd.Timestamp("2020-09-30 00:00:00")

# Filter out the excluded range
filtered_range = full_range[(full_range < exclude_start) | (full_range > exclude_end)]

# Convert to list of strings in yyyymmdd format
date_str_list = filtered_range.strftime("%Y%m%d %H:%M:%S").tolist()

r500 = r_regrid.sel(time=date_str_list).mean(dim=("latitude","longitude"))

In [4]:
dir = '/scratch/nf33/hk25_LSP/UK/MC/'
dir = '/scratch/nf33/hk25_LSP/Germany/Darwin/'
fname = 'RH500_3hr.nc'
fname = 'RH500_6hr.nc'

r500.to_netcdf(dir+fname)
print(f"Saved to ",dir+fname)

Saved to  /scratch/nf33/hk25_LSP/Germany/Darwin/RH500_6hr.nc
